In [1]:
using StatsPlots, Statistics, Distributions, Random

In [2]:
function generate_midpoints(n_bins)
    quantiles = Array{Float64}(undef, n_bins)
    quantile_midpoints = Array{Float64}(undef, n_bins)

    #generate the quantile midpoints
    for i = 1:n_bins
        quantiles[i] = i/n_bins
        quantile_midpoints[i] = i/n_bins - (1/2)*(1/n_bins)
    end

    return vec(quantiles), vec(quantile_midpoints)
end

generate_midpoints (generic function with 1 method)

In [3]:
function multiply_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] * list2[i]
    end
    
    return vec(sort(sum_list))
end

multiply_distributions (generic function with 1 method)

In [4]:
function re_bin(distribution, n_bins)
    rebinned = reshape(distribution, n_bins, n_bins)
    rebinned = mean(rebinned, dims = 1)
    
    return vec(rebinned)
end

re_bin (generic function with 1 method)

In [5]:
# takes a vector and scalar as input, computes element-wise division

function calculate_mean(distribution, n_bins)
    return sum(distribution .* 1/n_bins)
end

calculate_mean (generic function with 1 method)

In [6]:
# takes vector and two scalars as input, computes the empirical variance

function calculate_variance(distribution, calculated_mean, n_bins)
    return sum(distribution.^2 * (1/n_bins)) - calculated_mean^2
end

calculate_variance (generic function with 1 method)

In [7]:
function true_mean(μ, n_RVs)
    # this function assumes that all RVs are i.i.d.
    #    recommend editing for modularity later
    return μ^n_RVs
end

true_mean (generic function with 1 method)

In [8]:
function true_variance(μ, σ, n_RVs)
    # this function assumes that all RVs are i.i.d.
    #    recommend editing for modularity later
    variance = 1
    mean_prod_square = true_mean(μ^2, n_RVs)
    
    for i = 1:n_RVs
        # compute the second moment of the product distribution
        variance = variance * (σ^2 + μ^2)
    end
    
    # compute the second central moment
    variance = variance - mean_prod_square
    return variance
end

true_variance (generic function with 1 method)

In [62]:
n_bins = 4000
n_RVs = 10
μ = 6
σ = 3

# mean_prod = 1
# var_prod = 1

RV_list = Array{Distribution}(undef, n_RVs)

for i = 1:n_RVs
#     μ = rand(5:10)
#     σ = rand(1:6)
    
    temp_RV = Normal(μ, σ)
    RV_list[i] = temp_RV
    
#     mean_prod = mean_prod * μ
#     var_prod = var_prod * σ^2
end

In [63]:
quantiles, quantile_boundaries = generate_midpoints(n_bins)

final_RV = pop!(RV_list)
final_RV = quantile.(final_RV, quantile_boundaries)

for RV in RV_list
    temp_list = quantile.(RV, quantile_boundaries)
    
    final_RV = multiply_distributions(temp_list, final_RV)
    final_RV = re_bin(final_RV, n_bins) 
end

In [64]:
calc_mean = calculate_mean(final_RV, n_bins)
calc_var = calculate_variance(final_RV, calc_mean, n_bins)

true_mu = true_mean(μ, n_RVs)
true_var = true_variance(μ, σ, n_RVs)

println("Expected Additive Mean is: ", true_mu)
println("Calculated Additive Mean is: ", calc_mean)

println("Expected Additive Variance is: ", true_var)
println("Calculated Additive Variance is: ", calc_var)

Expected Additive Mean is: 60466176
Calculated Additive Mean is: 6.046617599999996e7
Expected Additive Variance is: 30394470475952649
Calculated Additive Variance is: 2.9677594031188616e16


In [65]:
true_var/calc_var

1.0241554771593229

In [58]:
true_var > calc_var

true